In [ ]:
import time
import math
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from main import nmf
np.random.seed(42)

In [ ]:
%matplotlib inline

In [ ]:
def zeros_mask(arr):
    m, n = arr.shape
    indices = np.random.choice(m * n, replace=False, size=int(m * n * 0.2))
    arr[np.unravel_index(indices, (m, n))] = 0
    return arr

def plot_scores(fscores, gscores, _lambda):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_yscale('log')
    ax.plot(fscores)
    ax.plot(gscores)
    ax.plot(fscores + _lambda * gscores)
    ax.legend(['f', 'g', 'total'])

def normalized_similarity(W_ins):
    r = W_ins.shape[1]
    res = np.ones(shape=(r, r)) * -1
    for i in range(r):
        for j in range(r):
            res[i, j] = np.linalg.norm(W_ins[:, i] - W_ins[:, j])
        res[i, :] = res[i, :] / sum(res[i, :])
    return res

def load_results(filename):
    data = np.load(filename)
    return data['Wb'], data['Hb'], data['Wl'], data['Hl'], data['fscores'], data['gscores'], data['_lambda']

In [ ]:
# mat = scipy.io.loadmat('urban/Urban.mat')
# X = mat['X']
# with open(f'urban/fullX.npz', 'wb') as fout:
#     np.savez_compressed(fout, X=X)

In [ ]:
# m, n = X.shape # (162, 94249)
# num_col = int(math.sqrt(n)) # 307

In [ ]:
# X3d = X.reshape(m, num_col, num_col, order='F') # order specified to match MATLAB

In [ ]:
# wavelength = 100
# plt.imshow(X3d[wavelength, :, :], cmap='gray')
# plt.colorbar()

### Small

In [ ]:
# img = X3d[wavelength, :, :].copy()
# img[285: 295, 120: 130] = 1000 # trees
# img[200: 210, 265: 275] = 1000 # grass
# img[20: 30, 200: 210] = 1000 # asphalt
# plt.imshow(img, cmap='gray')
# plt.colorbar()

In [ ]:
# trees3d = X3d[:, 285: 295, 120: 130] # (162, 10, 10)
# grass3d = X3d[:, 200: 210, 265: 275] # (162, 10, 10)
# asphalt3d = X3d[:, 20: 30, 200: 210] # (162, 10, 10)
# smallX3d = np.hstack([trees3d, grass3d, asphalt3d]) # (162, 30, 10)

In [ ]:
# plt.imshow(smallX3d[wavelength, :, :], cmap='gray')
# plt.colorbar()

In [ ]:
# smallX = smallX3d.reshape(m, -1, order='F') # (162, 300)
# with open(f'urban/smallX.npz', 'wb') as fout:
#     np.savez_compressed(fout, X=smallX)

### Medium

In [ ]:
# img = X3d[wavelength, :, :].copy()
# img[240: 290, 180: 230] = 1000 # trees
# plt.imshow(img, cmap='gray')
# plt.colorbar()

In [ ]:
# medX3d = X3d[:, 240: 290, 180: 230] # (162, 50, 50)
# medX = medX3d.reshape(m, -1, order='F') # (162, 2500)
# with open(f'urban/medX.npz', 'wb') as fout:
#     np.savez_compressed(fout, X=medX)

### Running tests

In [ ]:
def run_nmf_and_save(size, iterations, ranks, _lambdas):
    X = np.load(f'urban/{size}X.npz')['X']
    m, n = X.shape

    for rank in ranks:
        W_ini = zeros_mask(np.random.rand(m, rank))
        H_ini = zeros_mask(np.random.rand(rank, n))
        with open(f'urban/{size}_ini_r{rank}.npz', 'wb') as fout:
            np.savez_compressed(fout, W_ini=W_ini, H_ini=H_ini)

        for _lambda in _lambdas:
            start_time = time.time()
            Wb, Hb, Wl, Hl, fscores, gscores = nmf(X, W_ini.copy(), H_ini.copy(), _lambda=_lambda, itermax=iterations)
            runtime_min = (time.time() - start_time) / 60
            with open(f'urban/results/{size}_r{rank}_it{iterations}_l{str(_lambda).replace(".", "-")}.npz', 'wb') as fout:
                np.savez_compressed(fout, Wb=Wb, Hb=Hb, Wl=Wl, Hl=Hl, fscores=fscores, gscores=gscores, _lambda=_lambda)
            print(f'NMF for size={size}, rank={rank}, lambda={_lambda} in {runtime_min} minutes.')

In [ ]:
size = 'small'
iterations = 1000
ranks = [6, 8]
_lambdas = [0.25, 0.5, 1, 2]

run_nmf_and_save(size, iterations, ranks, _lambdas)

In [ ]:
size = 'med'
iterations = 1000
ranks = [8, 10]
_lambdas = [0.25, 0.5, 1]

run_nmf_and_save(size, iterations, ranks, _lambdas)

In [ ]:
size = 'full'
iterations = 500
ranks = [8, 10]
_lambdas = [0.25, 0.5, 1]

run_nmf_and_save(size, iterations, ranks, _lambdas)